In [1]:
# Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

In [2]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path("alzheimer_clean.csv")   
)

# Review the DataFrame
df.head()

,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,SleepQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,9.025679,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,7.151293,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,9.673574,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,8.392554,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,5.597238,...,6.045039,0,0,0.014691,0,0,1,1,0,0


In [3]:
y = df["Diagnosis"]
x = df.drop(columns="Diagnosis")

In [4]:
# Encode the categorical variables using get_dummies
X = pd.get_dummies(x)

In [5]:
# split original dataset X and y into a training set + temp set (test_size=0.2 for 20% of data going towards the temp set and 80% for training set)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1)

# split temp set into validation set + test set (50/50) 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# goal = train using 80% of original data, tune hyperparameters using the validation set + evaluate model performance on unseen data using test data

In [6]:
print(X_train.shape, X_val.shape, X_test.shape)

(1719, 32) (215, 32) (215, 32)


In [7]:
# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

# Transform the validation data using the scaler
X_val_scaled = X_scaler.transform(X_val)

Doing a Grid Search for n_neighours optimal value using cross-validation in Python with SciKit-Learn

In [45]:
# Define the parameter grid with the correct parameter name 'n_neighbors'
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Access the best parameters and print the best n_neighbors value
best_n_neighbors = grid_search.best_params_['n_neighbors']
print("Best n_neighbors value:", best_n_neighbors)

Best n_neighbors value: 11


In [46]:
# Define the parameter grid with the correct parameter name 'n_neighbors'
param_grid = {'n_neighbors': [11, 13, 15, 17, 19]}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Access the best parameters and print the best n_neighbors value
best_n_neighbors = grid_search.best_params_['n_neighbors']
print("Best n_neighbors value:", best_n_neighbors)

Best n_neighbors value: 19


Trying the "rule of thumb for n_neighbours" = square root of total number of samples in the dataset (1719 for training set = 41)

In [47]:
# Define the parameter grid with the correct parameter name 'n_neighbors'
param_grid = {'n_neighbors': [35, 37, 39, 41, 43]}

# Create a KNN classifier
knn = KNeighborsClassifier()

# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Access the best parameters and print the best n_neighbors value
best_n_neighbors = grid_search.best_params_['n_neighbors']
print("Best n_neighbors value:", best_n_neighbors)

Best n_neighbors value: 43


Keep n_neighbors = 41 to attempt at best balance between bias and variance

In [48]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=41)

In [49]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier(n_neighbors=41)

In [50]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.641860465116279


In [51]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [52]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.97      0.82       139
           1       0.85      0.29      0.43        76

    accuracy                           0.73       215
   macro avg       0.78      0.63      0.63       215
weighted avg       0.76      0.73      0.68       215



Normalization to scale features to a range between 0 and 1 (want to bring data to common scale)

In [53]:
from sklearn.preprocessing import MinMaxScaler

In [54]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled_norm = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled_norm = X_scaler.transform(X_test)

# Transform the validation data using the scaler
X_val_scaled_norm = X_scaler.transform(X_val)

In [55]:
from sklearn.pipeline import make_pipeline

# Create a pipeline with StandardScaler and KNN classifier
pipeline = make_pipeline(MinMaxScaler(), KNeighborsClassifier(n_neighbors=41))

# Fit the pipeline on the training data and make predictions
pipeline.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=41))])

In [56]:
# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=41)

In [57]:
# Train the model using the training data
knn.fit(X_train_scaled_norm, y_train)

KNeighborsClassifier(n_neighbors=41)

In [58]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.641860465116279


In [59]:
# Create predictions using the testing data
y_pred = knn.predict(X_test)

In [60]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      1.00      0.79       139
           1       0.00      0.00      0.00        76

    accuracy                           0.65       215
   macro avg       0.32      0.50      0.39       215
weighted avg       0.42      0.65      0.51       215



c:\Users\yumai\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yumai\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yumai\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Normalization and Pipeline Feature has the same validation accuracy, but lowers f1-score whilst increasing recall for [0], which decreasing for [1]. 

NO IMPROVEMENT WITH CHANGE IN WEIGHT BY DISTANCE (uniform and distance the same)

In [61]:
# Create a k-NN classifier with distance-based weighting
knn_distance = KNeighborsClassifier(n_neighbors=41, weights='uniform')

In [62]:
# Train the model using the training data
knn_distance.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=41)

In [64]:
validation_accuracy = knn_distance.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

# validation decreased with weight change

Validation Accuracy: 0.641860465116279


In [65]:
# Create predictions using the testing data
y_pred = knn_distance.predict(X_test_scaled)

In [66]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.97      0.82       139
           1       0.85      0.29      0.43        76

    accuracy                           0.73       215
   macro avg       0.78      0.63      0.63       215
weighted avg       0.76      0.73      0.68       215



Try lowered n_neighbors (=19)

In [67]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=19)

In [68]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=19)

In [69]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.6232558139534884


In [70]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [71]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.95      0.82       139
           1       0.79      0.34      0.48        76

    accuracy                           0.73       215
   macro avg       0.76      0.65      0.65       215
weighted avg       0.75      0.73      0.70       215



Tried even lower n_neighbors (11) = lowered accuracy, recall, precision and approximately the same validation score. 

In [72]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=11)

In [73]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=11)

In [74]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.6


In [75]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [76]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.91      0.79       139
           1       0.64      0.28      0.39        76

    accuracy                           0.69       215
   macro avg       0.67      0.59      0.59       215
weighted avg       0.68      0.69      0.65       215



In [77]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 19 
knn = KNeighborsClassifier(n_neighbors=31)

In [78]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=31)

In [79]:
validation_accuracy = knn.score(X_val, y_val)
print("Validation Accuracy:", validation_accuracy)

Validation Accuracy: 0.6325581395348837


In [80]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [81]:
# Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.95      0.82       139
           1       0.79      0.34      0.48        76

    accuracy                           0.73       215
   macro avg       0.76      0.65      0.65       215
weighted avg       0.75      0.73      0.70       215



Lowered validation accuracy for n_neighbors = 31. Keep n_neighbors =41. Low accuracy and averaged f1-score of 0.73 has urged consideration of deeper learning. 

TESTING DEEP LEARNING MODELS

In [9]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

# results promising; but attempt to improve 

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 320us/sample - loss: 0.7956 - accuracy: 0.5009 - val_loss: 0.7617 - val_accuracy: 0.5116
Epoch 2/50
1719/1719 [==============================] - 0s 41us/sample - loss: 0.6325 - accuracy: 0.6434 - val_loss: 0.6405 - val_accuracy: 0.6512
Epoch 3/50
1719/1719 [==============================] - 0s 46us/sample - loss: 0.5469 - accuracy: 0.7353 - val_loss: 0.5660 - val_accuracy: 0.6884
Epoch 4/50
1719/1719 [==============================] - 0s 47us/sample - loss: 0.4943 - accuracy: 0.7725 - val_loss: 0.5142 - val_accuracy: 0.7302
Epoch 5/50
1719/1719 [==============================] - 0s 41us/sample - loss: 0.4568 - accuracy: 0.7993 - val_loss: 0.4768 - val_accuracy: 0.7442
Epoch 6/50
1719/1719 [==============================] - 0s 58us/sample - loss: 0.4291 - accuracy: 0.8162 - val_loss: 0.4450 - val_accuracy: 0.7581
Epoch 7/50
1719/1719 [==============================] - 0s 45us/sample

First start w/ grid search to see if random neuron number was optimal

In [12]:
# Define the deep learning model function

def create_model(neurons=16):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=32))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'neurons': [8, 16, 32, 64]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'neurons': 8}
Best Score:  0.840619695186615


Test model using 8 neurons in layer

-----> optimized 8 neurons x 1 layer (50 epochs to test)

In [11]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 297us/sample - loss: 0.7310 - accuracy: 0.5666 - val_loss: 0.6866 - val_accuracy: 0.5953
Epoch 2/50
1719/1719 [==============================] - 0s 45us/sample - loss: 0.6729 - accuracy: 0.6283 - val_loss: 0.6410 - val_accuracy: 0.6512
Epoch 3/50
1719/1719 [==============================] - 0s 45us/sample - loss: 0.6270 - accuracy: 0.6830 - val_loss: 0.6019 - val_accuracy: 0.6744
Epoch 4/50
1719/1719 [==============================] - 0s 42us/sample - loss: 0.5866 - accuracy: 0.7115 - val_loss: 0.5673 - val_accuracy: 0.6744
Epoch 5/50
1719/1719 [==============================] - 0s 42us/sample - loss: 0.5497 - accuracy: 0.7411 - val_loss: 0.5359 - val_accuracy: 0.7070
Epoch 6/50
1719/1719 [==============================] - 0s 48us/sample - loss: 0.5161 - accuracy: 0.7615 - val_loss: 0.5088 - val_accuracy: 0.7302
Epoch 7/50
1719/1719 [==============================] - 0s 43us/sample

Decrease neurons to = 8 increase accuracy for model, and slight decrease in validation (both values now over 0.8; keep neurons = 8)

Checking optimization of epochs # using GridSearchCV

In [13]:
# Define the deep learning model function

def create_model(neurons=8):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=32))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'epochs': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'epochs': 20}
Best Score:  0.8429435849189758


Checking validation set using new parameters (optimized epochs=20 as well as neuron =8 in single layer)

In [14]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/20
1719/1719 [==============================] - 1s 294us/sample - loss: 0.6932 - accuracy: 0.6341 - val_loss: 0.6265 - val_accuracy: 0.6372
Epoch 2/20
1719/1719 [==============================] - 0s 40us/sample - loss: 0.6137 - accuracy: 0.6818 - val_loss: 0.5659 - val_accuracy: 0.6930
Epoch 3/20
1719/1719 [==============================] - 0s 41us/sample - loss: 0.5568 - accuracy: 0.7248 - val_loss: 0.5246 - val_accuracy: 0.7302
Epoch 4/20
1719/1719 [==============================] - 0s 41us/sample - loss: 0.5145 - accuracy: 0.7510 - val_loss: 0.4949 - val_accuracy: 0.7674
Epoch 5/20
1719/1719 [==============================] - 0s 41us/sample - loss: 0.4819 - accuracy: 0.7778 - val_loss: 0.4697 - val_accuracy: 0.7814
Epoch 6/20
1719/1719 [==============================] - 0s 41us/sample - loss: 0.4563 - accuracy: 0.7917 - val_loss: 0.4501 - val_accuracy: 0.7860
Epoch 7/20
1719/1719 [==============================] - 0s 41us/sample

Trying epochs=40 because epochs=20 is lower than 50 (balanced with validation) -- keep epochs =50

In [15]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=40, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/40
1719/1719 [==============================] - 1s 387us/sample - loss: 1.0041 - accuracy: 0.3828 - val_loss: 0.9456 - val_accuracy: 0.4372
Epoch 2/40
1719/1719 [==============================] - 0s 40us/sample - loss: 0.8308 - accuracy: 0.4642 - val_loss: 0.8011 - val_accuracy: 0.4605
Epoch 3/40
1719/1719 [==============================] - 0s 44us/sample - loss: 0.7197 - accuracy: 0.5590 - val_loss: 0.7059 - val_accuracy: 0.5581
Epoch 4/40
1719/1719 [==============================] - 0s 44us/sample - loss: 0.6450 - accuracy: 0.6318 - val_loss: 0.6413 - val_accuracy: 0.6558
Epoch 5/40
1719/1719 [==============================] - 0s 46us/sample - loss: 0.5897 - accuracy: 0.6905 - val_loss: 0.5914 - val_accuracy: 0.6930
Epoch 6/40
1719/1719 [==============================] - 0s 41us/sample - loss: 0.5473 - accuracy: 0.7243 - val_loss: 0.5520 - val_accuracy: 0.7023
Epoch 7/40
1719/1719 [==============================] - 0s 45us/sample

Increasing epochs further from 50 also lowers accuracy for both test and validation -- keep 50 

In [16]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/100
1719/1719 [==============================] - 1s 323us/sample - loss: 0.8805 - accuracy: 0.4287 - val_loss: 0.7711 - val_accuracy: 0.5163
Epoch 2/100
1719/1719 [==============================] - 0s 56us/sample - loss: 0.7780 - accuracy: 0.5137 - val_loss: 0.6949 - val_accuracy: 0.5767
Epoch 3/100
1719/1719 [==============================] - 0s 45us/sample - loss: 0.7097 - accuracy: 0.5858 - val_loss: 0.6436 - val_accuracy: 0.6186
Epoch 4/100
1719/1719 [==============================] - 0s 41us/sample - loss: 0.6573 - accuracy: 0.6376 - val_loss: 0.6063 - val_accuracy: 0.6977
Epoch 5/100
1719/1719 [==============================] - 0s 41us/sample - loss: 0.6131 - accuracy: 0.6882 - val_loss: 0.5745 - val_accuracy: 0.7535
Epoch 6/100
1719/1719 [==============================] - 0s 39us/sample - loss: 0.5727 - accuracy: 0.7266 - val_loss: 0.5453 - val_accuracy: 0.7814
Epoch 7/100
1719/1719 [==============================] - 0s 43us

Checking parameters neurons x optimizer for optimization

In [8]:
# Define the deep learning model function

def create_model(neurons=8, optimizer="adam"):
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=neurons, activation="relu", input_dim=32))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return nn_model

# Create a KerasClassifier based on the model function
nn_model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

# Define the grid of hyperparameters to search
param_grid = {'neurons': [8, 16, 32],'optimizer': ["adam", "SGD", "RMSprop"]}

# Perform grid search
grid = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best parameters and results
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

Best Parameters:  {'neurons': 8, 'optimizer': 'RMSprop'}
Best Score:  0.8499186396598816


In [9]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 336us/sample - loss: 0.7312 - accuracy: 0.5561 - val_loss: 0.7085 - val_accuracy: 0.5488
Epoch 2/50
1719/1719 [==============================] - 0s 43us/sample - loss: 0.6463 - accuracy: 0.6411 - val_loss: 0.6467 - val_accuracy: 0.5860
Epoch 3/50
1719/1719 [==============================] - 0s 49us/sample - loss: 0.5903 - accuracy: 0.6835 - val_loss: 0.6020 - val_accuracy: 0.6512
Epoch 4/50
1719/1719 [==============================] - 0s 41us/sample - loss: 0.5496 - accuracy: 0.7266 - val_loss: 0.5658 - val_accuracy: 0.7023
Epoch 5/50
1719/1719 [==============================] - 0s 48us/sample - loss: 0.5159 - accuracy: 0.7574 - val_loss: 0.5341 - val_accuracy: 0.7116
Epoch 6/50
1719/1719 [==============================] - 0s 48us/sample - loss: 0.4879 - accuracy: 0.7784 - val_loss: 0.5078 - val_accuracy: 0.7395
Epoch 7/50
1719/1719 [==============================] - 0s 49us/sample

neuron =8, epochs =50, optimizer = RMSprop gives the best balance between test accuracy vs. validation accuracy

Testing other activation functions

---> AF = tanh decreases accuracy

---> PReLU also decreases accuracy

In [10]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="tanh", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 395us/sample - loss: 0.7101 - accuracy: 0.5550 - val_loss: 0.7222 - val_accuracy: 0.5023
Epoch 2/50
1719/1719 [==============================] - 0s 48us/sample - loss: 0.6331 - accuracy: 0.6248 - val_loss: 0.6463 - val_accuracy: 0.6140
Epoch 3/50
1719/1719 [==============================] - 0s 43us/sample - loss: 0.5707 - accuracy: 0.7068 - val_loss: 0.5860 - val_accuracy: 0.7023
Epoch 4/50
1719/1719 [==============================] - 0s 48us/sample - loss: 0.5189 - accuracy: 0.7632 - val_loss: 0.5356 - val_accuracy: 0.7442
Epoch 5/50
1719/1719 [==============================] - 0s 75us/sample - loss: 0.4764 - accuracy: 0.7976 - val_loss: 0.4962 - val_accuracy: 0.7628
Epoch 6/50
1719/1719 [==============================] - 0s 49us/sample - loss: 0.4430 - accuracy: 0.8220 - val_loss: 0.4657 - val_accuracy: 0.7721
Epoch 7/50
1719/1719 [==============================] - 0s 56us/sample

In [11]:
from tensorflow.keras.layers import PReLU

# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, input_dim=32))
nn_model.add(PReLU())  # Adding PReLU activation function here

# Output layer for binary classification (1 neuron for 0 or 1)
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model with validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data and validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

ERROR! Session/line number was not unique in database. History logging moved to new session 452
Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 396us/sample - loss: 0.6863 - accuracy: 0.5672 - val_loss: 0.6383 - val_accuracy: 0.6744
Epoch 2/50
1719/1719 [==============================] - 0s 54us/sample - loss: 0.6170 - accuracy: 0.6847 - val_loss: 0.5919 - val_accuracy: 0.7395
Epoch 3/50
1719/1719 [==============================] - 0s 49us/sample - loss: 0.5687 - accuracy: 0.7493 - val_loss: 0.5556 - val_accuracy: 0.7395
Epoch 4/50
1719/1719 [==============================] - 0s 58us/sample - loss: 0.5279 - accuracy: 0.7789 - val_loss: 0.5226 - val_accuracy: 0.7442
Epoch 5/50
1719/1719 [==============================] - 0s 50us/sample - loss: 0.4923 - accuracy: 0.8040 - val_loss: 0.4914 - val_accuracy: 0.7581
Epoch 6/50
1719/1719 [==============================] - 0s 53us/sample - loss: 0.4605 - accuracy: 0.8150 - val_loss: 0.46

Testing addition of more hidden layers

---> adding a hidden layer decreased accuracy and increased loss

In [12]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))
nn_model.add(Dense(units=8, activation='relu'))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 662us/sample - loss: 0.7791 - accuracy: 0.4090 - val_loss: 0.7078 - val_accuracy: 0.5163
Epoch 2/50
1719/1719 [==============================] - 0s 78us/sample - loss: 0.7022 - accuracy: 0.5102 - val_loss: 0.6681 - val_accuracy: 0.6186
Epoch 3/50
1719/1719 [==============================] - 0s 75us/sample - loss: 0.6657 - accuracy: 0.6353 - val_loss: 0.6459 - val_accuracy: 0.6837
Epoch 4/50
1719/1719 [==============================] - 0s 76us/sample - loss: 0.6402 - accuracy: 0.6946 - val_loss: 0.6243 - val_accuracy: 0.6930
Epoch 5/50
1719/1719 [==============================] - 0s 76us/sample - loss: 0.6135 - accuracy: 0.7208 - val_loss: 0.5994 - val_accuracy: 0.7163
Epoch 6/50
1719/1719 [==============================] - 0s 76us/sample - loss: 0.5822 - accuracy: 0.7574 - val_loss: 0.5705 - val_accuracy: 0.7395
Epoch 7/50
1719/1719 [==============================] - 0s 75us/sample

Optimizing learning speeds!

---> LR = 0.001 decreases accuracy 

---> LR = 0.005 increases accuracy

---> LR = 0.0001 decreases accuracy

---> LR = 0.01 decreases accuracy

In [11]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 331us/sample - loss: 0.7469 - accuracy: 0.5462 - val_loss: 0.7051 - val_accuracy: 0.6000
Epoch 2/50
1719/1719 [==============================] - 0s 50us/sample - loss: 0.6578 - accuracy: 0.6131 - val_loss: 0.6351 - val_accuracy: 0.6512
Epoch 3/50
1719/1719 [==============================] - 0s 42us/sample - loss: 0.5971 - accuracy: 0.6643 - val_loss: 0.5842 - val_accuracy: 0.6791
Epoch 4/50
1719/1719 [==============================] - 0s 43us/sample - loss: 0.5504 - accuracy: 0.7167 - val_loss: 0.5456 - val_accuracy: 0.7209
Epoch 5/50
1719/1719 [==============================] - 0s 38us/sample - loss: 0.5133 - accuracy: 0.7464 - val_loss: 0.5135 - val_accuracy: 0.7442
Epoch 6/50
1719/1719 [==============================] - 0s 38us/sample - loss: 0.4822 - accuracy: 0.7725 - val_loss: 0.4867 - val_accuracy: 0.7721
Epoch 7/50
1719/1719 [==============================] - 0s 37us/sample

In [12]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.005)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 337us/sample - loss: 0.6269 - accuracy: 0.6789 - val_loss: 0.4485 - val_accuracy: 0.8140
Epoch 2/50
1719/1719 [==============================] - 0s 39us/sample - loss: 0.4482 - accuracy: 0.8092 - val_loss: 0.3842 - val_accuracy: 0.8419
Epoch 3/50
1719/1719 [==============================] - 0s 40us/sample - loss: 0.3868 - accuracy: 0.8406 - val_loss: 0.3600 - val_accuracy: 0.8279
Epoch 4/50
1719/1719 [==============================] - 0s 36us/sample - loss: 0.3646 - accuracy: 0.8505 - val_loss: 0.3577 - val_accuracy: 0.8233
Epoch 5/50
1719/1719 [==============================] - 0s 41us/sample - loss: 0.3505 - accuracy: 0.8517 - val_loss: 0.3527 - val_accuracy: 0.8279
Epoch 6/50
1719/1719 [==============================] - 0s 40us/sample - loss: 0.3420 - accuracy: 0.8551 - val_loss: 0.3584 - val_accuracy: 0.8279
Epoch 7/50
1719/1719 [==============================] - 0s 41us/sample

In [14]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 342us/sample - loss: 0.5135 - accuracy: 0.7627 - val_loss: 0.4122 - val_accuracy: 0.7814
Epoch 2/50
1719/1719 [==============================] - 0s 40us/sample - loss: 0.3842 - accuracy: 0.8429 - val_loss: 0.3733 - val_accuracy: 0.7953
Epoch 3/50
1719/1719 [==============================] - 0s 39us/sample - loss: 0.3594 - accuracy: 0.8522 - val_loss: 0.3793 - val_accuracy: 0.8047
Epoch 4/50
1719/1719 [==============================] - 0s 42us/sample - loss: 0.3472 - accuracy: 0.8650 - val_loss: 0.3647 - val_accuracy: 0.8093
Epoch 5/50
1719/1719 [==============================] - 0s 41us/sample - loss: 0.3369 - accuracy: 0.8633 - val_loss: 0.3633 - val_accuracy: 0.8326
Epoch 6/50
1719/1719 [==============================] - 0s 47us/sample - loss: 0.3272 - accuracy: 0.8726 - val_loss: 0.3755 - val_accuracy: 0.8326
Epoch 7/50
1719/1719 [==============================] - 0s 43us/sample

In [13]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = nn_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 346us/sample - loss: 0.7561 - accuracy: 0.5492 - val_loss: 0.7017 - val_accuracy: 0.6000
Epoch 2/50
1719/1719 [==============================] - 0s 41us/sample - loss: 0.7425 - accuracy: 0.5544 - val_loss: 0.6917 - val_accuracy: 0.6093
Epoch 3/50
1719/1719 [==============================] - 0s 47us/sample - loss: 0.7297 - accuracy: 0.5649 - val_loss: 0.6821 - val_accuracy: 0.6140
Epoch 4/50
1719/1719 [==============================] - 0s 47us/sample - loss: 0.7175 - accuracy: 0.5736 - val_loss: 0.6730 - val_accuracy: 0.6140
Epoch 5/50
1719/1719 [==============================] - 0s 46us/sample - loss: 0.7058 - accuracy: 0.5817 - val_loss: 0.6645 - val_accuracy: 0.6233
Epoch 6/50
1719/1719 [==============================] - 0s 43us/sample - loss: 0.6945 - accuracy: 0.5910 - val_loss: 0.6560 - val_accuracy: 0.6465
Epoch 7/50
1719/1719 [==============================] - 0s 43us/sample

In [8]:
# Define the deep learning model 
final32_model = tf.keras.models.Sequential()
final32_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=32))

# Output layer/binary classification so 1 neuron for 0 or 1
final32_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.005)

# Compile the Sequential model together and customize metrics
final32_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model w/ validation dataset
fit32_model = final32_model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

# Evaluate the model using the test data + validation data
model_loss, model_accuracy = final32_model.evaluate(X_test_scaled,y_test,verbose=2)
model_loss_val, model_accuracy_val = final32_model.evaluate(X_val_scaled, y_val, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Validation Set - Loss: {model_loss_val}, Accuracy: {model_accuracy_val}")

Train on 1719 samples, validate on 215 samples
Epoch 1/50
1719/1719 [==============================] - 1s 391us/sample - loss: 0.5940 - accuracy: 0.6958 - val_loss: 0.5280 - val_accuracy: 0.7070
Epoch 2/50
1719/1719 [==============================] - 0s 48us/sample - loss: 0.4555 - accuracy: 0.7964 - val_loss: 0.4447 - val_accuracy: 0.7860
Epoch 3/50
1719/1719 [==============================] - 0s 44us/sample - loss: 0.3942 - accuracy: 0.8319 - val_loss: 0.4071 - val_accuracy: 0.7907
Epoch 4/50
1719/1719 [==============================] - 0s 46us/sample - loss: 0.3675 - accuracy: 0.8505 - val_loss: 0.3886 - val_accuracy: 0.8233
Epoch 5/50
1719/1719 [==============================] - 0s 39us/sample - loss: 0.3519 - accuracy: 0.8633 - val_loss: 0.3806 - val_accuracy: 0.8140
Epoch 6/50
1719/1719 [==============================] - 0s 40us/sample - loss: 0.3430 - accuracy: 0.8633 - val_loss: 0.3776 - val_accuracy: 0.8279
Epoch 7/50
1719/1719 [==============================] - 0s 43us/sample

In [9]:
final32_model.save("NN_AllFeatures_model.h5")

In [10]:
final32_model.save("NN_AllFeatures_model")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: NN_AllFeatures_model\assets


In [13]:
# Calculate Spearman's rank correlation coefficients with the target variable
spearman_corr = df.corr(method='spearman')['Diagnosis'].abs().sort_values(ascending=False)

# Print the ranked features based on Spearman's correlation coefficients
print("Ranked Features based on Spearman's Correlation Coefficients:")
print(spearman_corr)

# Select the top 'k' features based on your criteria
# For example, selecting the top 5 features
top_features = spearman_corr.index[:10].tolist()
print("Top Features selected:")
print(top_features)

Ranked Features based on Spearman's Correlation Coefficients:
Diagnosis                    1.000000
FunctionalAssessment         0.366687
ADL                          0.330450
MemoryComplaints             0.306742
MMSE                         0.236271
BehavioralProblems           0.224350
SleepQuality                 0.056069
EducationLevel               0.043325
CholesterolHDL               0.042542
Hypertension                 0.035080
FamilyHistoryAlzheimers      0.032900
CholesterolLDL               0.032010
Diabetes                     0.031508
CardiovascularDisease        0.031490
BMI                          0.026402
Disorientation               0.024648
CholesterolTriglycerides     0.023072
HeadInjury                   0.021411
Gender                       0.020975
PersonalityChanges           0.020627
Confusion                    0.019186
Ethnicity                    0.017744
SystolicBP                   0.015822
DifficultyCompletingTasks    0.009069
DietQuality               